In [54]:
from sklearn.neural_network import MLPClassifier
from sklearn import svm, preprocessing
from sklearn.naive_bayes import GaussianNB, BernoulliNB

import sklearn.metrics as sklm

from sklearn import linear_model
#from statsmodels.api import datasets
from sklearn import datasets ## Get dataset from sklearn
import sklearn.model_selection as ms
import sklearn.metrics as sklm
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import numpy.random as nr
from dateutil.relativedelta import relativedelta

import math

%matplotlib inline

In [55]:
AdvWorksCustsFinal = pd.read_csv('AWCustFinal.csv',index_col='CustomerID')

In [56]:
AdvWorksCustsFinal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16404 entries, 11000 to 18982
Data columns (total 14 columns):
Unnamed: 0              16404 non-null int64
CountryRegionName       16404 non-null object
Education               16404 non-null object
Occupation              16404 non-null object
Gender                  16404 non-null object
MaritalStatus           16404 non-null object
HomeOwnerFlag           16404 non-null int64
NumberCarsOwned         16404 non-null int64
NumberChildrenAtHome    16404 non-null int64
TotalChildren           16404 non-null int64
YearlyIncome            16404 non-null int64
BikeBuyer               16404 non-null int64
ageat1998               16404 non-null int64
AveMonthSpendLog        16404 non-null float64
dtypes: float64(1), int64(8), object(5)
memory usage: 1.9+ MB


In [57]:
advWorksCusts = pd.read_csv('AdvWorksCusts.csv')
#print(advWorksCusts.CustomerID.unique().shape)
advWorksCusts.drop_duplicates(subset = 'CustomerID', keep = 'last', inplace = True)

In [58]:
aw_AveMonthSpend = pd.read_csv('AW_AveMonthSpend.csv')
#print(aw_AveMonthSpend.CustomerID.unique().shape)
aw_AveMonthSpend.drop_duplicates(subset = 'CustomerID', keep = 'last', inplace = True)
print(advWorksCusts.shape)
aw_BikeBuyer = pd.read_csv('AW_BikeBuyer.csv')
#print(aw_BikeBuyer.CustomerID.unique().shape)
aw_BikeBuyer.drop_duplicates(subset = 'CustomerID', keep = 'last', inplace = True)
print(advWorksCusts.shape)

(16404, 23)
(16404, 23)


In [59]:
WorksCustsFinal = pd.merge(advWorksCusts, aw_AveMonthSpend, on='CustomerID')
WorksCustsFinal = pd.merge(WorksCustsFinal, aw_BikeBuyer,on='CustomerID')
print(WorksCustsFinal.shape)
print(WorksCustsFinal.CustomerID.unique().shape)

(16404, 25)
(16404,)


In [60]:
WorksCustsFinal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16404 entries, 0 to 16403
Data columns (total 25 columns):
CustomerID              16404 non-null int64
Title                   88 non-null object
FirstName               16404 non-null object
MiddleName              9453 non-null object
LastName                16404 non-null object
Suffix                  2 non-null object
AddressLine1            16404 non-null object
AddressLine2            273 non-null object
City                    16404 non-null object
StateProvinceName       16404 non-null object
CountryRegionName       16404 non-null object
PostalCode              16404 non-null object
PhoneNumber             16404 non-null object
BirthDate               16404 non-null object
Education               16404 non-null object
Occupation              16404 non-null object
Gender                  16404 non-null object
MaritalStatus           16404 non-null object
HomeOwnerFlag           16404 non-null int64
NumberCarsOwned         16404

In [61]:
WorksCustsFinal.columns
WorksCustsFinal.drop(columns=['Title', 'MiddleName', 'Suffix', 'AddressLine2'],axis=1,inplace=True)

In [62]:
WorksCustsFinal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16404 entries, 0 to 16403
Data columns (total 21 columns):
CustomerID              16404 non-null int64
FirstName               16404 non-null object
LastName                16404 non-null object
AddressLine1            16404 non-null object
City                    16404 non-null object
StateProvinceName       16404 non-null object
CountryRegionName       16404 non-null object
PostalCode              16404 non-null object
PhoneNumber             16404 non-null object
BirthDate               16404 non-null object
Education               16404 non-null object
Occupation              16404 non-null object
Gender                  16404 non-null object
MaritalStatus           16404 non-null object
HomeOwnerFlag           16404 non-null int64
NumberCarsOwned         16404 non-null int64
NumberChildrenAtHome    16404 non-null int64
TotalChildren           16404 non-null int64
YearlyIncome            16404 non-null int64
AveMonthSpend          

In [63]:
WorksCustsFinal['ageat1998'] = [1998 - int(str(dt).split('-')[0]) for dt in WorksCustsFinal['BirthDate'] ]

In [64]:
WorksCustsFinal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16404 entries, 0 to 16403
Data columns (total 22 columns):
CustomerID              16404 non-null int64
FirstName               16404 non-null object
LastName                16404 non-null object
AddressLine1            16404 non-null object
City                    16404 non-null object
StateProvinceName       16404 non-null object
CountryRegionName       16404 non-null object
PostalCode              16404 non-null object
PhoneNumber             16404 non-null object
BirthDate               16404 non-null object
Education               16404 non-null object
Occupation              16404 non-null object
Gender                  16404 non-null object
MaritalStatus           16404 non-null object
HomeOwnerFlag           16404 non-null int64
NumberCarsOwned         16404 non-null int64
NumberChildrenAtHome    16404 non-null int64
TotalChildren           16404 non-null int64
YearlyIncome            16404 non-null int64
AveMonthSpend          

In [65]:
WorksCustsFinal.drop(WorksCustsFinal[['FirstName','LastName','AddressLine1','City','StateProvinceName','PostalCode','PhoneNumber','BirthDate']],axis=1,inplace=True)

In [66]:
#AdvWorksCustsFinal.drop(['Unnamed: 0'],axis = 1,inplace=True,)

In [75]:
# Labels
Labels = np.array(WorksCustsFinal['AveMonthSpend'])
Labels.shape

(16404,)

In [68]:
# Encode categorical data
def encode_string(cat_features):
    
    ## First encode the strings to numeric categories
    enc = preprocessing.LabelEncoder()
    enc.fit(cat_features)
    enc_cat_features = enc.transform(cat_features)
    
    ## Now, apply one hot encoding
    ohe = preprocessing.OneHotEncoder()
    encoded = ohe.fit(enc_cat_features.reshape(-1,1))
    return encoded.transform(enc_cat_features.reshape(-1,1)).toarray()

categorical_columns = ['Education','Occupation',
                       'Gender','MaritalStatus','HomeOwnerFlag']

Features = encode_string(WorksCustsFinal['CountryRegionName'])

for col in categorical_columns:
    temp = encode_string(WorksCustsFinal[col])
    Features = np.concatenate([Features, temp], axis = 1)
    
print(Features.shape)
print(Features[:2, :])

(16404, 22)
[[1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0.]]


In [69]:
numerical_columns = numerical_columns = ['NumberCarsOwned', 'NumberChildrenAtHome', 'TotalChildren', 'YearlyIncome', 'ageat1998']

In [70]:
# Add numerical feature
Features = np.concatenate([Features, np.array(WorksCustsFinal[numerical_columns])], axis = 1)
Features[:2,:]
print(Features.shape)

(16404, 27)


In [76]:
## Randomly sample cases to create independent training and test data
nr.seed(9988)
indx = range(Features.shape[0])
indx = ms.train_test_split(indx, test_size = 0.3)
X_train = Features[indx[0],:]
y_train = np.ravel(Labels[indx[0]])
X_test = Features[indx[1],:]
y_test = np.ravel(Labels[indx[1]])

In [77]:
scaler = preprocessing.StandardScaler().fit(X_train[:,23:])
X_train[:,23:] = scaler.transform(X_train[:,23:])
X_test[:,23:] = scaler.transform(X_test[:,23:])

In [78]:
#Construct Linear regression model

lin_mod = linear_model.LinearRegression(fit_intercept = False)
lin_mod.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=1, normalize=False)

In [79]:
ads_test = pd.read_csv('AW_test.csv')
ads_test.head()

,CustomerID,Title,FirstName,MiddleName,LastName,Suffix,AddressLine1,AddressLine2,City,StateProvinceName,...,BirthDate,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome
0,18988,NaN,Courtney,A,Baker,NaN,8727 Buena Vista Ave.,NaN,Fremont,California,...,1/5/1945,Bachelors,Management,F,S,0,2,0,5,86931
1,29135,NaN,Adam,C,Allen,NaN,3491 Cook Street,NaN,Haney,British Columbia,...,10/4/1964,Bachelors,Skilled Manual,M,M,1,2,2,4,100125
2,12156,NaN,Bonnie,NaN,Raji,NaN,359 Pleasant Hill Rd,NaN,Burbank,California,...,1/12/1934,Graduate Degree,Management,F,M,1,2,0,4,103985
3,13749,NaN,Julio,C,Alonso,NaN,8945 Euclid Ave.,NaN,Burlingame,California,...,9/22/1958,Graduate Degree,Skilled Manual,M,M,1,0,0,4,127161
4,27780,NaN,Christy,A,Andersen,NaN,"42, boulevard Tremblay",NaN,Dunkerque,Nord,...,3/19/1965,High School,Manual,F,M,1,1,2,2,21876


In [80]:
# Calculate age of customer based on birthdate
def calculate_age(end):
    r = relativedelta(pd.to_datetime('now'), pd.to_datetime(end)) 
    return '{}'.format(r.years)


ads_test['Age'] = ads_test['BirthDate'].apply(calculate_age)

# Convert age to integer
ads_test['Age'] = ads_test['Age'].astype('int64')

In [81]:
ads_totest = ads_test[['CountryRegionName','Education','Occupation','Gender',
                      'MaritalStatus','HomeOwnerFlag','NumberCarsOwned',
                      'NumberChildrenAtHome','TotalChildren','YearlyIncome',
                      'Age']]

ads_totest.head()

,CountryRegionName,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,Age
0,United States,Bachelors,Management,F,S,0,2,0,5,86931,74
1,Canada,Bachelors,Skilled Manual,M,M,1,2,2,4,100125,54
2,United States,Graduate Degree,Management,F,M,1,2,0,4,103985,85
3,United States,Graduate Degree,Skilled Manual,M,M,1,0,0,4,127161,60
4,France,High School,Manual,F,M,1,1,2,2,21876,53


In [82]:
# Encode categorical data
def encode_string(cat_features):
    
    ## First encode the strings to numeric categories
    enc = preprocessing.LabelEncoder()
    enc.fit(cat_features)
    enc_cat_features = enc.transform(cat_features)
    
    ## Now, apply one hot encoding
    ohe = preprocessing.OneHotEncoder()
    encoded = ohe.fit(enc_cat_features.reshape(-1,1))
    return encoded.transform(enc_cat_features.reshape(-1,1)).toarray()

categorical_columns = ['Education','Occupation',
                       'Gender','MaritalStatus','HomeOwnerFlag']

Features = encode_string(ads_totest['CountryRegionName'])

for col in categorical_columns:
    temp = encode_string(ads_totest[col])
    Features = np.concatenate([Features, temp], axis = 1)
    
print(Features.shape)

(500, 22)


In [83]:
# Add numerical feature
Features = np.concatenate([Features, np.array(ads_totest[['NumberCarsOwned',
                                                        'NumberChildrenAtHome',
                                                        'TotalChildren',
                                                        'YearlyIncome','Age']])], axis = 1)

print(Features.shape)

(500, 27)


In [84]:
X_test = Features
X_test[:,23:] = scaler.transform(X_test[:,23:])

In [85]:
scores = lin_mod.predict(X_test)
print(scores)

[ 40.16992188 104.2734375   46.44335938  86.19140625  58.21289062
  40.57226562  93.2265625  124.66796875 100.3359375   53.9375
  56.81787109  48.44140625  69.99804688  44.15283203  35.34912109
  48.94921875  83.42578125  70.45703125 109.50976562  56.91992188
  66.13085938  73.453125   146.94921875  82.53125     52.734375
  72.78515625  84.18359375 114.63916016  74.8203125   59.57617188
  66.59765625  78.14453125  37.04296875  69.35546875 103.52734375
 102.02734375 146.68164062  90.04296875  56.359375    84.546875
  44.265625    78.08007812  80.12695312  46.12890625  55.8046875
  73.59765625  59.07617188  84.63671875 114.09375     77.921875
  79.3125      92.875       78.81445312  62.3046875   44.40332031
  73.53515625  55.18457031  72.65771484  61.81445312  67.77539062
  43.734375    62.890625    88.37207031  80.82421875  41.69726562
  78.48046875  79.58203125 130.73632812  63.76953125 104.14404297
  83.31640625  66.1171875   90.66943359  42.01757812  63.19042969
  79.76660156 113.022

In [86]:
result_regression = pd.DataFrame(scores, index = ads_test['CustomerID'],columns=['AveMonthSpend'])

result_regression
result_regression.to_csv('result_regression.csv')